In [ ]:
from pip._internal import main as pipmain
import subprocess
# !conda install -c conda-forge gdcm -y
subprocess.getoutput('cp ../input/gdcm-conda-install/gdcm.tar .')
subprocess.getoutput('tar -xvzf gdcm.tar')
subprocess.getoutput('conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2')
pipmain(['install', '../input/pretrained-models/pretrained-models.pytorch-master'])
pipmain(['install', '../input/geffnet/gen-efficientnet-pytorch/gen-efficientnet-pytorch-master'])
print("done")

In [ ]:
import warnings
warnings.filterwarnings(action='once')
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from misclib import *
from processdata import *
from pytorchmodels import *
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
import copy
from torchvision import transforms
import PIL.Image
from sklearn.metrics import roc_auc_score
import torchvision.transforms.functional as TF
from types import MethodType
#import sandesh
import pydicom
import os

In [ ]:
params=json_to_parameters('../input/rsna2020-config/kaggle_config.json')
do_full=False
do_z=False
num_folds=5
SEED=220
params=json_to_parameters('../input/rsna2020-config/kaggle_config.json')
do_full=False
do_z=False
num_folds=5
SEED=220
models=[{'inference_fold':0,
        'model_type':'tf_efficientnet_b5_ns',
        'base_version':1,
        'output_size':11,
        'pool':False,
        'name_tamplate':'image_{}',
        'transformer':[{'transformer_version':2,
        'num_heads':2,
        'num_layers':4,
        'ffdim':2048,
        'max_len':128,
        'tta':15,
        'noise':0.3,
        'transformer_tamplate_add':''}]},
        {'inference_fold':2,
        'model_type':'tf_efficientnet_b5_ns',
        'base_version':'pd',
        'output_size':11,
        'pool':True,
        'name_tamplate':'image_{}',
        'transformer':[{'transformer_version':'pdn03f02t128',
        'num_heads':2,
        'num_layers':4,
        'ffdim':3072,
        'max_len':128,
        'tta':15,
        'noise':0.3,
        'transformer_tamplate_add':''}]},
        {'inference_fold':1,
        'model_type':'tf_efficientnet_b5_ns',
        'base_version':'pdint',
        'output_size':12,
        'pool':True,
        'name_tamplate':'image_{}',
        'transformer':[{'transformer_version':'pdint128n03f02',
        'num_heads':4,
        'num_layers':4,
        'max_len':128,
        'tta':15,
        'noise':0.3,
        'ffdim':3072,
        'transformer_tamplate_add':''}]},
        {'inference_fold':0,
        'model_type':'tf_efficientnet_b3_ns',
        'base_version':'slice',
        'output_size':12,
        'pool':False,
        'name_tamplate':'image_{}',
        'transformer':[{'transformer_version':'slice',
        'num_heads':4,
        'num_layers':6,
        'ffdim':2048,
        'max_len':128,
        'tta':15,
        'noise':0.3,
        'transformer_tamplate_add':''}]}]
#         'model_type':'tf_efficientnet_b5_ns',
#         'base_version':1,
#         'output_size':11,
#         'pool':False,
#         'name_tamplate':'image_{}',
#         'transformer':[{'transformer_version':2,
#         'num_heads':2,
#         'num_layers':4,
#         'ffdim':2048,
#         'transformer_tamplate_add':''},
#         {'transformer_version':2,
#         'num_heads':4,
#         'num_layers':4,
#         'ffdim':2048,
#         'transformer_tamplate_add':''},
#         {'transformer_version':2,
#         'num_heads':2,
#         'num_layers':4,
#         'ffdim':3072,
#         'transformer_tamplate_add':''}]}]
for mp in models:
    for t in mp['transformer']:
        t['transformer_name_tamplate']='transformer'+t['transformer_tamplate_add']+f'_{t["num_heads"]}_{t["num_layers"]}_{t["ffdim"]}_'+'{}'

seed_add = 123
num_tta=1
# transformer_tta=15
# max_len=128
cols=  ['rv_lv_ratio_gte_1',
        'rv_lv_ratio_lt_1',
        'leftsided_pe',
        'chronic_pe',
        'negative_exam_for_pe',
        'rightsided_pe',
        'acute_and_chronic_pe',
        'central_pe',
        'indeterminate']
cols_dict=dict(zip(cols,range(len(cols))))
if params.platform=='myserver':
    device = device_by_name('RTX')
    numworkers=12
    base_batch=32
    trans_batch=32
    img_type='pkl'
else:
    device=('cuda:0')
    numworkers=2
    base_batch=16
    trans_batch=32
    img_type='dicom'
torch.cuda.set_device(device)

In [ ]:
# test_imgs = glob.glob(f"{params.path.test}*/*/*")
# errored_files=[]
# col = ['sop_instance_uid','study_instance_uid','series_instance_uid','instance_number','file_name']
# df=dicom_get_df_data(test_imgs,image_stats=False,errored_files=errored_files,key_list=col)    
# gp=df[['study_instance_uid','series_instance_uid','instance_number']].groupby(['series_instance_uid'])
# gp_slice=gp.instance_number.agg(min_slice='min',max_slice='max').reset_index()
# df=df.merge(gp_slice,on='series_instance_uid',how='left')
# df['rel_slice']=(df.instance_number-df.min_slice)/(df.max_slice-df.min_slice)
# df.rename(columns = {'sop_instance_uid':'SOPInstanceUID', 'study_instance_uid':'StudyInstanceUID', 
#                           'series_instance_uid':'SeriesInstanceUID'}, inplace = True)
# print (f'create df is done size of df {df.shape}')

In [ ]:
from os import path
if path.exists('../input/rsna-str-pulmonary-embolism-detection/train') and not do_full:
    df=pd.read_csv(params.path.data+'test.csv').head(3000)
else:
    df=pd.read_csv(params.path.data+'test.csv')
# df.to_csv('public_test.csv', index = False)
file_names=df.StudyInstanceUID+'/'+df.SeriesInstanceUID+'/'+df.SOPInstanceUID+'.dcm'

pos=np.zeros(df.shape[0],dtype=np.long)
print ('start reading dicom for pos')
load_fail=0
for i,f in enumerate(file_names):
    try:
        img_dicom = pydicom.read_file(params.path.test+f)
        pos[i] = img_dicom.InstanceNumber if not do_z else float(img_dicom.ImagePositionPatient[-1]) 
    except Exception as e:
        print (filename,e)
        pos[i]=0
        load_fail=load_fail+1

if not do_z:
    df['instance_number']=pos
    gp=df[['StudyInstanceUID','SeriesInstanceUID','instance_number']].groupby(['SeriesInstanceUID'])
    gp_slice=gp.instance_number.agg(min_slice='min',max_slice='max').reset_index()
    df=df.merge(gp_slice,on='SeriesInstanceUID',how='left')
    df['rel_slice']=(df.instance_number-df.min_slice)/(df.max_slice-df.min_slice)
else:
    df['z']=pos
    gp=df[['StudyInstanceUID','SeriesInstanceUID','z']].groupby(['SeriesInstanceUID'])
    gp_slice=gp.z.agg(z_min='min',z_max='max').reset_index()
    df=df.merge(gp_slice,on='SeriesInstanceUID',how='left')
    df['rel_slice']=(df.z-df.z_min)/1200
    df=df.sort_values('z').reset_index(drop=True)
    df['instance_number']=df[['SeriesInstanceUID','SOPInstanceUID']].groupby('SeriesInstanceUID')['SOPInstanceUID'].cumcount()+1


df.head()

In [ ]:
# define transforms
transform=transforms.Compose([RandomResizedCropTransform(scale=(0.7, 1.3), ratio=(0.7, 1.3)),
                              RotateTransform(25),
                              RandomFlipTransform(0.5,0.5),
                              RandomChangeMeanStdTransform(30,0.1),
                              CutoutTransform(0.5,0.2),
                              CutoutTransform(0.25,0.3),
                              ResizeTransform(512,512)])

transform_val=transforms.Compose([ResizeTransform(512,512)])


In [ ]:
class MinDif():
    def __init__(self,d):
        self.d=d
    def __call__(self,key):
        return abs(self.d.__getitem__(key)-0.5).min()
    


In [ ]:
studydict={key:[] for key in df.loc[:,'StudyInstanceUID']}
image_dict={key:[] for key in df.loc[:,'SOPInstanceUID']}
studydict_res={key:[] for key in df.loc[:,'StudyInstanceUID']}
image_dict_res={key:[] for key in df.loc[:,'SOPInstanceUID']}
sub_df=df.copy()

In [ ]:
seed_add=0
ak=[3,3,6,6]
bk=[60,60,150,150]
for kk,mp in enumerate(models): #(fold,model_type,model_version) in zip(inference_folds,model_types,base_versions):
    # create features
    image_reader=ImageReader(params.path.test,image_type='dicom',return_pos=False)
    ds=ImageDataset(image_reader,sub_df,transform=transform_val)
    #model_version=base_version
    fold=mp['inference_fold']
    model_type=mp['model_type']
    model_version=mp['base_version']
    name_tamplate=mp['name_tamplate']
    output_size=mp['output_size']
    torch.manual_seed(SEED+fold+seed_add)
    np.random.seed(SEED+fold+seed_add)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    model = get_model(model_type,output_size=output_size, feature_size =256,amp=True,pretrained=False,pool=mp['pool']).to(device)
    name=params.model_format.format(model_type,name_tamplate.format(SEED),model_version,fold)
    print (name)
    model.load_state_dict(torch.load(f'{params.path.models}{name}',map_location=device))
    model.last_linear=Noop()
    model=model.eval()
    ds=ImageDataset(image_reader,sub_df,transform=transform_val,file_ext='.dcm')
    print(f'starting with features for {name}')
    dl=D.DataLoader(ds,num_workers=numworkers,batch_size=base_batch,shuffle=False)
    features=np.zeros((len(ds),256),dtype=np.float32)
    k=0
    with torch.no_grad():
#             tk0 = notebook.tqdm(dl) if verbose else dl
        for i,batch in enumerate(notebook.tqdm(dl)):
            features[k:k+batch[0].shape[0]]= model(batch[0].to(device)).to('cpu').detach().numpy()  
            k=k+batch[0].shape[0]
    featuress=[features]
    print (f'featuress len {len(featuress)} just added {featuress[-1].shape}')
    if num_tta>1:
        ds=ImageDataset(image_reader,sub_df,transform=transform,file_ext='.dcm')
        dl=D.DataLoader(ds,num_workers=numworkers,batch_size=base_batch,shuffle=False)
        for i in range(num_tta-1):
            features=np.zeros((len(ds),256),dtype=np.float32)
            k=0
            with torch.no_grad():
    #             tk0 = notebook.tqdm(dl) if verbose else dl
                for i,batch in enumerate(notebook.tqdm(dl)):
                    features[k:k+batch[0].shape[0]]= model(batch[0].to(device)).to('cpu').detach().numpy()  
                    k=k+batch[0].shape[0]
            featuress.append(features)
        print (f'ffeaturess len {len(featuress)} just added {featuress[-1].shape}')
    features=np.stack(featuress,0)
#         with open(params.path.features+(name.split('.')[0]+'.pkl'),'wb') as f:
#             pickle.dump(features,f,protocol=4)
#         del features,featuress
#         print (f'features are saved for {name}')

    preds=[]
    inds=[]
    all_patients=sub_df.SeriesInstanceUID.unique()
    fold=mp['inference_fold']
    model_type=mp['model_type']
    base_version=mp['base_version']
    name_tamplate=mp['name_tamplate']
#         basic_name=params.model_format.format(model_type,name_tamplate.format(SEED),base_version,fold).split('.')[0]+'.pkl'
#         with open(params.path.features+basic_name,'rb') as f:
#             features=torch.tensor(pickle.load(f),dtype=torch.float32)
#         print(f'features loaded {basic_name}')
    for tr in mp['transformer']:
        ffdim=tr['ffdim']
        num_layers=tr['num_layers']
        num_heads=tr['num_heads']
        transformer_name_tamplate=tr['transformer_name_tamplate']
        transformer_version=tr['transformer_version']
        max_len=tr['max_len']
        transformer_tta=tr['tta']
        fnoise=tr['noise']
        ds=PatientFeaturesDataset(torch.tensor(features,dtype=torch.float32),sub_df,all_patients,max_len=max_len,rand_split=(transformer_tta>1),rep=1,fnoise=fnoise)
        model = get_transformer_model(dim_feedforward=ffdim,n_heads=num_heads,n_encoders=num_layers).to(device)
        name=params.model_format.format(model_type,transformer_name_tamplate.format(SEED),transformer_version,fold)
        model.load_state_dict(torch.load(f'{params.path.models}{name}',map_location=device))
        model=model.eval()
        print('start predicting')
        for t in range(transformer_tta):
            ds.reset()
            dl=D.DataLoader(ds,num_workers=numworkers,batch_size=trans_batch,shuffle=False)
            p=np.zeros((len(ds),max_len,14),dtype=np.float32)
            ri=np.zeros((len(ds),max_len),dtype=np.long)
            k=0
            with torch.no_grad():
    #             tk0 = notebook.tqdm(dl) if verbose else dl
                for i,batch in enumerate(dl):
                    y = model(batch[0].to(device),batch[1].to(device),batch[2].to(device),mask=(batch[1]==-1).to(device))
                    p[k:k+batch[0].shape[0]]= y.to('cpu').detach().numpy()
                    ri[k:k+batch[0].shape[0]] = batch[-1]
                    k=k+batch[0].shape[0]

            preds.append(p)
            inds.append(ri)
            print(f'{t} {name} finished')

    #ensemble results
    for t,p in enumerate(preds):
        idxs=inds[t]
        for i,pl in enumerate(p):
            studydict[sub_df.loc[idxs[i,0],'StudyInstanceUID']].append(pl[-1,5:])
            for j,ind in enumerate(idxs[i][idxs[i]>=0]):
                image_dict[sub_df.loc[ind,'SOPInstanceUID']].append(pl[j,0])
    for key in studydict_res.keys():
        studydict_res[key]=torch.sigmoid(torch.tensor(np.mean(studydict[key],0))).numpy()
    for key in image_dict_res.keys():
        image_dict_res[key]=torch.sigmoid(torch.tensor(np.mean(image_dict[key],0))).item()

    needs2=sorted(studydict_res,key = MinDif(studydict_res))[:len(studydict_res)//ak[kk]]
    needs2_image=df[df.SOPInstanceUID.isin(sorted(image_dict_res,key =image_dict_res.__getitem__ )[:len(image_dict_res)//bk[kk]])].StudyInstanceUID.unique().tolist()
    sub_df=df[(df.StudyInstanceUID.isin(needs2)) | (df.StudyInstanceUID.isin(needs2_image))].reset_index(drop=True).copy()


In [ ]:
# [len(studydict[key]) for key in studydict.keys()]

In [ ]:
#check consistency 
def find_conflicts(studydict,image_dict):
    conflict=[]
    image_conflict=[]
    h=np.arange(9)
    for key in studydict.keys():
        images=df[df.StudyInstanceUID==key].SOPInstanceUID.values
        image_values=np.array([image_dict[image_key] for image_key in images])
        if studydict[key][cols_dict['negative_exam_for_pe']]>0.5:
            image_conflict.extend(images[image_values>0.5])
            if studydict[key][h!=cols_dict['negative_exam_for_pe']].max()>0.5:
                conflict.append(key)
                continue
        elif studydict[key][cols_dict['indeterminate']]>0.5:
            image_conflict.extend(images[image_values>0.5])
            if studydict[key][h!=cols_dict['indeterminate']].max()>0.5:
                conflict.append(key)
                continue
                
        else:
            if image_values.max()<=0.5:
                    conflict.append(key)
                    continue            
            if (studydict[key][cols_dict['rv_lv_ratio_gte_1']]>0.5)==(studydict[key][cols_dict['rv_lv_ratio_lt_1']]>0.5):
                conflict.append(key)
                continue
            if (studydict[key][[cols_dict['chronic_pe'],cols_dict['acute_and_chronic_pe']]]>0.5).sum()>1:
                    conflict.append(key)
                    continue
            if studydict[key][[cols_dict['central_pe'],
                              cols_dict['rightsided_pe'],
                             cols_dict['leftsided_pe']]].max()<=0.5:
                    conflict.append(key)
                    continue
    return conflict,image_conflict
        


In [ ]:
c=find_conflicts(studydict_res,image_dict_res)
print(len(c[0]),len(c[1]))
h=np.arange(9)

for key in studydict_res.keys():
    if studydict_res[key][cols_dict['negative_exam_for_pe']]>0.5:
        if studydict_res[key][h!=cols_dict['negative_exam_for_pe']].max()>studydict_res[key][cols_dict['negative_exam_for_pe']]:
            studydict_res[key][cols_dict['negative_exam_for_pe']]=0.4999
        else:
            studydict_res[key][h!=cols_dict['negative_exam_for_pe']]=np.clip(studydict_res[key][h!=cols_dict['negative_exam_for_pe']],0,0.4999)
for key in studydict_res.keys():
    if studydict_res[key][cols_dict['indeterminate']]>0.5:
        if studydict_res[key][h!=cols_dict['indeterminate']].max()>studydict_res[key][cols_dict['indeterminate']]:
            studydict_res[key][cols_dict['indeterminate']]=0.4999
        else:
            studydict_res[key][h!=cols_dict['indeterminate']]=np.clip(studydict_res[key][h!=cols_dict['indeterminate']],0,0.4999)

for key in studydict_res.keys():
    if studydict_res[key][cols_dict['negative_exam_for_pe']]>0.5 or studydict_res[key][cols_dict['indeterminate']]>0.5:
        images=df[df.StudyInstanceUID==key].SOPInstanceUID.values
        for image_key in images:
            image_dict_res[image_key] = min(image_dict_res[image_key],0.4999)
    else:
        images=df[df.StudyInstanceUID==key].SOPInstanceUID.values
        am=np.argmax(np.array([image_dict_res[image_key] for image_key in images]))
        image_dict_res[images[am]]=0.5001

for key in studydict_res.keys():
    if studydict_res[key][cols_dict['negative_exam_for_pe']]<=0.5 and studydict_res[key][cols_dict['indeterminate']]<=0.5:
        if (studydict_res[key][cols_dict['rv_lv_ratio_gte_1']]>0.5)==(studydict_res[key][cols_dict['rv_lv_ratio_lt_1']]>0.5):
            if studydict_res[key][cols_dict['rv_lv_ratio_gte_1']]>0.5:
                if studydict_res[key][cols_dict['rv_lv_ratio_gte_1']]>studydict_res[key][cols_dict['rv_lv_ratio_lt_1']]:
                    studydict_res[key][cols_dict['rv_lv_ratio_lt_1']]=0.4999
                else:
                    studydict_res[key][cols_dict['rv_lv_ratio_gte_1']]=0.4999
            else:
                if studydict_res[key][cols_dict['rv_lv_ratio_gte_1']]<studydict_res[key][cols_dict['rv_lv_ratio_lt_1']]:
                    studydict_res[key][cols_dict['rv_lv_ratio_lt_1']]=0.5001
                else:
                    studydict_res[key][cols_dict['rv_lv_ratio_gte_1']]=0.5001
                
        if (studydict_res[key][[cols_dict['chronic_pe'],cols_dict['acute_and_chronic_pe']]]>0.5).sum()>1:
            if studydict_res[key][cols_dict['chronic_pe']]>studydict_res[key][cols_dict['acute_and_chronic_pe']]:
                studydict_res[key][cols_dict['acute_and_chronic_pe']]=0.4999
            else:
                studydict_res[key][cols_dict['chronic_pe']]=0.4999
        if studydict_res[key][[cols_dict['central_pe'],
                          cols_dict['rightsided_pe'],
                         cols_dict['leftsided_pe']]].max()<=0.5:
            am = np.argmax(studydict_res[key][[cols_dict['central_pe'],
                          cols_dict['rightsided_pe'],
                         cols_dict['leftsided_pe']]])
            studydict_res[key][[cols_dict['central_pe'],
                          cols_dict['rightsided_pe'],
                         cols_dict['leftsided_pe']][am]]=0.501
c=find_conflicts(studydict_res,image_dict_res)
print(len(c[0]),len(c[1]))
for i in c[0]:
    print(i,studydict_res[i])
for i in c[1]:
    print (i,image_dict_res[i])

In [ ]:
def check_consistency(sub, test):    
    '''
    Checks label consistency and returns the errors
    
    Args:
    sub   = submission dataframe (pandas)
    test  = test.csv dataframe (pandas)
    '''
    
    # EXAM LEVEL
    for i in test['StudyInstanceUID'].unique():
        df_tmp = sub.loc[sub.id.str.contains(i, regex = False)].reset_index(drop = True)
        df_tmp['StudyInstanceUID'] = df_tmp['id'].str.split('_').str[0]
        df_tmp['label_type']       = df_tmp['id'].str.split('_').str[1:].apply(lambda x: '_'.join(x))
        del df_tmp['id']
        if i == test['StudyInstanceUID'].unique()[0]:
            df = df_tmp.copy()
        else:
            df = pd.concat([df, df_tmp], axis = 0)
    df_exam = df.pivot(index = 'StudyInstanceUID', columns = 'label_type', values = 'label')
    
    # IMAGE LEVEL
    df_image = sub.loc[sub.id.isin(test.SOPInstanceUID)].reset_index(drop = True)
    df_image = df_image.merge(test, how = 'left', left_on = 'id', right_on = 'SOPInstanceUID')
    df_image.rename(columns = {"label": "pe_present_on_image"}, inplace = True)
    del df_image['id']
    
    # MERGER
    df = df_exam.merge(df_image, how = 'left', on = 'StudyInstanceUID')
    ids    = ['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']
    labels = [c for c in df.columns if c not in ids]
    df = df[ids + labels]
    
    # SPLIT NEGATIVE AND POSITIVE EXAMS
    df['positive_images_in_exam'] = df['StudyInstanceUID'].map(df.groupby(['StudyInstanceUID']).pe_present_on_image.max())
    df_pos = df.loc[df.positive_images_in_exam >  0.5]
    df_neg = df.loc[df.positive_images_in_exam <= 0.5]
    
    # CHECKING CONSISTENCY OF POSITIVE EXAM LABELS
    rule1a = df_pos.loc[((df_pos.rv_lv_ratio_lt_1  >  0.5)  & 
                         (df_pos.rv_lv_ratio_gte_1 >  0.5)) | 
                        ((df_pos.rv_lv_ratio_lt_1  <= 0.5)  & 
                         (df_pos.rv_lv_ratio_gte_1 <= 0.5))].reset_index(drop = True)
    rule1a['broken_rule'] = '1a'
    rule1b = df_pos.loc[(df_pos.central_pe    <= 0.5) & 
                        (df_pos.rightsided_pe <= 0.5) & 
                        (df_pos.leftsided_pe  <= 0.5)].reset_index(drop = True)
    rule1b['broken_rule'] = '1b'
    rule1c = df_pos.loc[(df_pos.acute_and_chronic_pe > 0.5) & 
                        (df_pos.chronic_pe           > 0.5)].reset_index(drop = True)
    rule1c['broken_rule'] = '1c'
    rule1d = df_pos.loc[(df_pos.indeterminate        > 0.5) | 
                        (df_pos.negative_exam_for_pe > 0.5)].reset_index(drop = True)
    rule1d['broken_rule'] = '1d'

    # CHECKING CONSISTENCY OF NEGATIVE EXAM LABELS
    rule2a = df_neg.loc[((df_neg.indeterminate        >  0.5)  & 
                         (df_neg.negative_exam_for_pe >  0.5)) | 
                        ((df_neg.indeterminate        <= 0.5)  & 
                         (df_neg.negative_exam_for_pe <= 0.5))].reset_index(drop = True)
    rule2a['broken_rule'] = '2a'
    rule2b = df_neg.loc[(df_neg.rv_lv_ratio_lt_1     > 0.5) | 
                        (df_neg.rv_lv_ratio_gte_1    > 0.5) |
                        (df_neg.central_pe           > 0.5) | 
                        (df_neg.rightsided_pe        > 0.5) | 
                        (df_neg.leftsided_pe         > 0.5) |
                        (df_neg.acute_and_chronic_pe > 0.5) | 
                        (df_neg.chronic_pe           > 0.5)].reset_index(drop = True)
    rule2b['broken_rule'] = '2b'
    
    # MERGING INCONSISTENT PREDICTIONS
    errors = pd.concat([rule1a, rule1b, rule1c, rule1d, rule2a, rule2b], axis = 0)
    
    # OUTPUT
    print('Found', len(errors), 'inconsistent predictions')
    return errors

In [ ]:
# image_dict.update({f'{key}_{col}': studydict[key][i] for key in studydict.keys() for i,col in enumerate(cols)})
# res_df=pd.DataFrame(data={'id':list(image_dict.keys()),'label':list(image_dict.values())})

# res_df.to_csv('submission.csv', index = False)
# res_df.head(10)
# res_df.tail(10)


In [ ]:
# create submission file
if len(c[0])+len(c[1])==0:
    image_dict_res.update({f'{key}_{col}': studydict_res[key][i] for key in studydict_res.keys() for i,col in enumerate(cols)})
    res_df=pd.DataFrame(data={'id':list(image_dict_res.keys()),'label':list(image_dict_res.values())})
    errors=check_consistency(res_df, df)
    if len(errors)==0 and load_fail==0:
        res_df.to_csv('submission.csv', index = False)
        res_df.head(10)
        res_df.tail(10)
else:
    image_dict_res.update({f'{key}_{col}': studydict_res[key][i] for key in studydict_res.keys() for i,col in enumerate(cols)})
    res_df=pd.DataFrame(data={'id':list(image_dict_res.keys()),'label':list(image_dict_res.values())})
    res_df.to_csv('no_submission.csv', index = False)
    res_df.head(10)
    res_df.tail(10)

    print(c)